In [46]:
import os
import re

from estnltk import EnvelopingBaseSpan
from estnltk import Text, Layer, Annotation, EnvelopingSpan

In [47]:
with open("vallakohus_esimene\Harju_Hageri_Kohila_id20604_1883a.ann", encoding="UTF8") as annotated_file:
    annotation = annotated_file.read()
with open("vallakohus_esimene\Harju_Hageri_Kohila_id20604_1883a.txt", encoding="UTF8") as txt:
    textfile = txt.read()

In [48]:
print(annotation)

T1	Isik 34 42	J. Krais
T2	Isik 56 72	Juhan Lesserberg
T3	Isik 76 86	Juhan Asse
T4	Isik 146 158	J.Lesserberg
T5	Isik 190 200	Juhan Asse
T6	Isik 231 241	Lesserberg
T7	Isik 357 373	Juhan Lesserberg
T8	Isik 418 428	Juhan Asse
T9	Isik 584 600	Juhan Lesserberg
T10	Isik 740 750	Juhan Asse



In [49]:
# converting the text form .txt file into an EstNLTK Text object and giving it the "words" layer
text = Text(textfile)
text = text.tag_layer(['words'])

In [50]:
# creating new layers to add to the Text object later on
gold_ner_layer = Layer(name="gold_ner", text_object=text, attributes=['nertag'])
gold_wordner_layer = Layer(name="gold_wordner", text_object=text, attributes=['nertag'], parent="words")


In [51]:
# creating a dictionary to hold the annotation data, the dictionary looks as follows:
# {"TRIGGER": ['NERTAG INDEX1 INDEX2', 'NAMED-ENTITY'], ...}
annotation_dictionary = {}

splitted_annotation = annotation.split("\n")
for row in splitted_annotation:
    splitted_row = row.split("\t")
    if (len(splitted_row) == 3):
        trigger = splitted_row[0]
        location = splitted_row[1]
        entity = splitted_row[2]
        annotation_dictionary[trigger] = [location, entity]
    else:
        print("Rea '" + row + "' lugemisel tekkis probleem.")

Rea '' lugemisel tekkis probleem.


In [52]:
# looping through every key and value in dictionary to add to the layer
for key in annotation_dictionary:    
    value = annotation_dictionary.get(key)
    print(key, value)

T1 ['Isik 34 42', 'J. Krais']
T2 ['Isik 56 72', 'Juhan Lesserberg']
T3 ['Isik 76 86', 'Juhan Asse']
T4 ['Isik 146 158', 'J.Lesserberg']
T5 ['Isik 190 200', 'Juhan Asse']
T6 ['Isik 231 241', 'Lesserberg']
T7 ['Isik 357 373', 'Juhan Lesserberg']
T8 ['Isik 418 428', 'Juhan Asse']
T9 ['Isik 584 600', 'Juhan Lesserberg']
T10 ['Isik 740 750', 'Juhan Asse']


In [53]:
dictionary_for_wordner = {} #the dictionary for wordner
T = 1 #just a trigger number

for key in annotation_dictionary:
    name = []
    value = annotation_dictionary.get(key)
    ner, loc = value[0].split(" ")[0], (value[0].split(" ")[1], value[0].split(" ")[2])
    preceding_newlines = text.text[0:int(loc[0])].count("\n") #counting preceding newlines to deduct from the indexes
    
    startIndex = int(loc[0]) - int(preceding_newlines) #starting index form the .ann file
    endIndex = int(loc[1]) - int(preceding_newlines) #ending index from the .ann file
    
    for i in range(0, len(text.words) - 1): #looping through every word in text.words
        if text.words[i].start == startIndex: #if the startindex is same in the .ann file and in the loop, add an new span 
            if text.words[i].end == endIndex:
                base_span = EnvelopingBaseSpan([text.words[i].base_span])
                name = [text.words[i]] #adding the name element(s) to a list to use in the wordner dictionary later on
            else:
                if text.words[i+1].end == endIndex: #if the next word's endindex is the same as the endindex in the .ann file, add a new span
                    name = [text.words[i], text.words[i+1]] 
                    base_span = EnvelopingBaseSpan([s.base_span for s in name])
                else:
                    for j in range(len(value[0].split(" ") - 1)):
                        name.append(text.words[i+j])
                base_span = EnvelopingBaseSpan([s.base_span for s in name])
                
            new_span = EnvelopingSpan(base_span, layer=gold_ner_layer)
            
            if ner == "Isik":
                new_span.add_annotation(Annotation(new_span, nertag="PER"))
                print(f"ISIK: {new_span} on lisatud.")
                for k in range(0, len(name)):
                    if k == 0:
                        dictionary_for_wordner["B-PER" + " " + str(T)] = i
                        continue
                    else:
                        dictionary_for_wordner["I-PER" + " " + str(T)] = i+k
            if ner == "KO_koht" or ner == "KO_org":
                new_span.add_annotation(Annotation(new_span, nertag="LOC"))
                new_span.add_annotation(Annotation(new_span, nertag="ORG"))
                print(f"LOC_ORG: {new_span} on lisatud.")
                for k in range(0, len(name)):
                    if k == 0:
                        dictionary_for_wordner["B-LOC" + " " + str(T)] = i
                        dictionary_for_wordner["B-ORG" + " " + str(T)] = i
                        continue
                        
                    else:
                        dictionary_for_wordner["I-LOC" + " " + str(T)] = i+k
                        dictionary_for_wordner["I-ORG" + " " + str(T)] = i+k
            if ner == "Koht":
                new_span.add_annotation(Annotation(new_span, nertag="LOC"))
                print(f"LOC: {new_span} on lisatud.")
                for k in range(0, len(name)):
                    if k == 0:
                        dictionary_for_wordner["B-LOC" + " " + str(T)] = i
                        continue
                    else:
                        dictionary_for_wordner["I-LOC" + " " + str(T)] = i+k
            if ner == "Org":
                new_span.add_annotation(Annotation(new_span, nertag="ORG"))
                print(f"ORG: {new_span} on lisatud.")
                for k in range(0, len(name)):
                    if k == 0:
                        dictionary_for_wordner["B-ORG" + " " + str(T)] = i
                        continue
                    else:
                        dictionary_for_wordner["I-ORG" + " " + str(T)] = i+k
            if ner == "Muu" or ner == "Teadmata" or ner == "ese":
                new_span.add_annotation(Annotation(new_span, nertag="MISC"))
                print(f"MISC: {new_span} on lisatud.")
                for k in range(0, len(name)):
                    if k == 0:
                        dictionary_for_wordner["B-MISC" + " " + str(T)] = i
                        continue
                    else:
                        dictionary_for_wordner["I-MISC" + " " + str(T)] = i+k
            gold_ner_layer.add_span(new_span)
            T += 1
            break
text.add_layer(gold_ner_layer)
#text.add_layer(gold_wordner_layer)

ISIK: EnvelopingSpan(['J. Krais'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['J.Lesserberg'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Lesserberg'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]) on lisatud.
ISIK: EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]) on lisatud.


In [54]:
for key in dictionary_for_wordner:
    val = dictionary_for_wordner.get(key)
    key_nertag = key.split(" ")[0]
    for i in range(0, len(text.words) - 1):
        if i == val:
            new_span = Span(base_span=text.words[i].base_span, layer=gold_wordner_layer)
            new_span.add_annotation(Annotation(new_span, nertag=str(key_nertag)))
            gold_wordner_layer.add_span(new_span)
            print("siin.")
            break
        else:
            new_span_o = Span(base_span=text.words[i].base_span, layer=gold_wordner_layer)
            new_span_o.add_annotation(Annotation(new_span_o, nertag="O"))
            print("siin.")
            gold_wordner_layer.add_span(new_span_o)
            
text.add_layer(gold_wordner_layer)

In [55]:
text.gold_ner

Layer(name='gold_ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['J. Krais'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]),
EnvelopingSpan(['J.Lesserberg'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Lesserberg'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Lesserberg'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Juhan', 'Asse'], [{'nertag': 'PER'}])])

In [56]:
text['gold_wordner']

Layer(name='gold_wordner', attributes=('nertag',), spans=SL[])